# Graft Loss Feature Importance with Monte Carlo Cross-Validation

**Study Replication:** Publication-quality replication with Monte Carlo cross-validation (configurable 100–1000 splits)  
**Updated:** November 21, 2025  
**Hardware:** Optimized for 32-core EC2 instance (1TB RAM)  
**Validation:** Proper evaluation on unseen test data

## Key Changes from Original

✅ **Monte Carlo Cross-Validation** – up to 1000 random 75/25 train/test splits (100-split runs used for faster iteration)  
✅ **Stratified Sampling** - Maintains event distribution  
✅ **Parallel Processing** - Fast execution with furrr/future (≈30 workers)  
✅ **95% Confidence Intervals** - Narrow, precise estimates (tighter with more splits)  
✅ **Realistic C-Indexes** - Expected range 0.70-0.85

## Methodology

This notebook implements the feature selection methodology from the original bcjaeger/graft-loss study:

1. Load data for pediatric heart transplant outcomes
2. Define three time periods:
   - **Original**: 2010-2019 (matches publication)
   - **Full**: 2010-2024 (all available data)
   - **Full No COVID**: 2010-2024 excluding 2020-2023
3. For each period and method:
   - Create 100–1000 stratified train/test splits (75/25)
   - Train model on training set
   - Evaluate on unseen test set
   - Aggregate results across splits
4. Calculate C-index with 95% CI
5. Extract top 20 features

## Expected Runtime (per full 3-period × 3-method run)

- **100 splits (current default):**
  - Local (4 cores): ~2–4 hours (NOT RECOMMENDED)
  - Workstation (16 cores): ~1–2 hours
  - EC2 (32 cores, 1TB RAM): ~1–2 hours ✅ **RECOMMENDED FOR DEVELOPMENT**
- **1000 splits (extended / publication-level):**
  - Local (4 cores): 8–12+ hours (NOT RECOMMENDED)
  - Workstation (16 cores): ~8–16 hours
  - EC2 (32 cores, 1TB RAM): ~10–20 hours ✅ **RECOMMENDED FOR FINAL RESULTS**


## 1. Setup and Configuration

Load required packages and configure parallel processing.


In [1]:
# Check R version
R.version.string

# Load required packages
library(here)
library(dplyr)
library(readr)
library(survival)
library(ranger)
library(aorsf)
library(catboost)
library(tidyr)
library(purrr)
library(tibble)
library(janitor)
library(haven)
library(riskRegression)
library(prodlim)
library(rsample)    # For MC-CV
library(furrr)      # For parallel processing
library(future)     # For parallel backend
library(progressr)  # For progress bars

cat("✓ All packages loaded successfully\n")


[1] "R version 4.4.3 (2025-02-28)"

here() starts at /home/pgx3874/graft-loss


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


riskRegression version 2025.09.17

Loading required package: future


Attaching package: ‘future’


The following object is masked from ‘package:survival’:

    cluster




✓ All packages loaded successfully


In [2]:
# ============================================================
# DEBUG/TEST MODE - Quick testing before full 100-split run
# ============================================================
# Set DEBUG_MODE = TRUE for quick testing (5 splits, ~2-5 min)
# Set DEBUG_MODE = FALSE for full analysis (100 splits, ~1-2 hours on EC2)

DEBUG_MODE <- FALSE  # Change to TRUE for quick test

if (DEBUG_MODE) {
  cat("\n")
  cat("╔════════════════════════════════════════════════════════════════╗\n")
  cat("║                    🔍 DEBUG MODE ENABLED                       ║\n")
  cat("╚════════════════════════════════════════════════════════════════╝\n")
  cat("\n")
  cat("Quick test configuration:\n")
  cat("  • MC-CV Splits: 5 (instead of 100)\n")
  cat("  • Period: Original only (2010-2019)\n")
  cat("  • Trees: Reduced (RSF: 100, AORSF: 50)\n")
  cat("  • Expected time: 2-5 minutes\n")
  cat("  • Purpose: Verify everything works before full run\n")
  cat("\n")
  cat("To run full analysis, set DEBUG_MODE = FALSE\n")
  cat("\n")
}

# Configuration
n_predictors <- 20                                        # Top 20 features
n_trees_rsf <- if (DEBUG_MODE) 100 else 500              # RSF trees (reduced in debug)
n_trees_aorsf <- if (DEBUG_MODE) 50 else 100             # AORSF trees (reduced in debug)
horizon <- 1                                              # 1-year prediction
n_mc_splits <- if (DEBUG_MODE) 5 else 100                # MC-CV splits (5 for debug, 100 for full)
train_prop <- 0.75                                        # 75% training, 25% testing

# Set up parallel processing - EC2 Optimized (32 cores, 1TB RAM)
# Use 30 out of 32 cores (leave 2 for system)
n_workers <- as.integer(Sys.getenv("N_WORKERS", "0"))
if (n_workers < 1) {
  total_cores <- parallel::detectCores()
  n_workers <- max(1, total_cores - 2)
  cat(sprintf("Auto-detected %d cores, using %d workers\n", total_cores, n_workers))
}

cat(sprintf("Setting up parallel processing with %d workers...\n", n_workers))
cat(sprintf("Expected speedup: %dx faster than single core\n", round(n_workers * 0.8)))

# Increase future.globals.maxSize for large MC-CV splits object
# With 1TB RAM on EC2, we can handle large transfers
options(future.globals.maxSize = 20 * 1024^3)  # 20 GB limit (plenty for 100 splits)
cat("Set future.globals.maxSize to 20 GB\n")

plan(multisession, workers = n_workers)

# Create output directory
output_dir <- here("feature_importance", "outputs")
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

cat("Output directory:", output_dir, "\n")
cat(sprintf("MC-CV Configuration: %d splits, %.0f/%.0f train/test split\n", 
            n_mc_splits, train_prop * 100, (1 - train_prop) * 100))
flush.console()


Auto-detected 32 cores, using 30 workers
Setting up parallel processing with 30 workers...
Expected speedup: 24x faster than single core
Set future.globals.maxSize to 20 GB
Output directory: /home/pgx3874/graft-loss/feature_importance/outputs 
MC-CV Configuration: 100 splits, 75/25 train/test split


## 2. Helper Functions

Define functions for data preparation, C-index calculation, and prediction.


In [3]:
# Prepare modeling data with leakage prevention
prepare_modeling_data <- function(data) {
  # Find time and status columns
  time_col <- intersect(c("time", "outcome_int_graft_loss", "int_graft_loss"), names(data))[1]
  status_col <- intersect(c("status", "outcome_graft_loss", "graft_loss"), names(data))[1]
  
  if (is.na(time_col) || is.na(status_col)) {
    stop("Cannot find time/status columns")
  }
  
  # Rename to standard names
  if (time_col != "time") data <- data %>% rename(time = !!time_col)
  if (status_col != "status") data <- data %>% rename(status = !!status_col)
  
  # Exclude leakage variables and identifier columns
  exclude_exact <- c(
    "ID", "ptid_e", "int_dead", "int_death", "graft_loss", "txgloss", "death", "event",
    "dpricaus", "deathspc", "concod", "age_death", "dlist", "txpl_year",
    "rrace_b", "rrace_a", "rrace_ai", "rrace_pi", "rrace_o", "rrace_un", "race",
    "patsupp", "pmorexam", "papooth", "pacuref", "pishltgr",
    "pathero", "pcadrec", "pcadrem", "pdiffib", "cpathneg",
    "dcardiac", "dneuro", "dreject", "dsecaccs", "dpriaccs",
    "dconmbld", "dconmal", "dconcard", "dconneur", "dconrej",
    "dmajbld", "dmalcanc"
  )
  
  exclude_prefixes <- c("dtx_", "cc_", "dcon", "dpri", "dsec", "dmaj", "sd")
  
  exclude_by_prefix <- character(0)
  for (prefix in exclude_prefixes) {
    exclude_by_prefix <- c(exclude_by_prefix, 
                           names(data)[startsWith(names(data), prefix)])
  }
  
  exclude_all <- unique(c(exclude_exact, exclude_by_prefix))
  data <- data %>% select(-any_of(exclude_all))
  
  # Median imputation for numeric variables
  numeric_vars <- names(data)[sapply(data, is.numeric) & names(data) != "time" & names(data) != "status"]
  for (var in numeric_vars) {
    if (any(is.na(data[[var]]))) {
      median_val <- median(data[[var]], na.rm = TRUE)
      data[[var]][is.na(data[[var]])] <- median_val
    }
  }
  
  # Mode imputation for categorical variables
  categorical_vars <- names(data)[sapply(data, function(x) is.factor(x) | is.character(x))]
  for (var in categorical_vars) {
    if (any(is.na(data[[var]]))) {
      mode_val <- names(sort(table(data[[var]]), decreasing = TRUE))[1]
      data[[var]][is.na(data[[var]])] <- mode_val
    }
  }
  
  # Remove constant columns
  constant_cols <- names(data)[sapply(data, function(x) length(unique(na.omit(x))) <= 1)]
  if (length(constant_cols) > 0) {
    data <- data %>% select(-any_of(constant_cols))
  }
  
  # Convert character to factor
  data <- data %>% mutate(across(where(is.character), as.factor))
  
  return(data)
}

cat("✓ prepare_modeling_data() defined\n")


✓ prepare_modeling_data() defined


In [4]:
# C-index calculation
calculate_cindex <- function(time, status, risk_scores, horizon = NULL) {
  valid_idx <- !is.na(time) & !is.na(status) & !is.na(risk_scores) &
               is.finite(time) & is.finite(risk_scores) & time > 0
  
  time   <- as.numeric(time[valid_idx])
  status <- as.numeric(status[valid_idx])
  risk   <- as.numeric(risk_scores[valid_idx])
  
  n <- length(time)
  events <- sum(status == 1)
  
  if (n < 10 || events < 1 || length(unique(risk)) == 1) {
    return(list(cindex_td = NA_real_, cindex_ti = NA_real_))
  }
  
  # Time-independent C-index (Harrell's)
  num_conc_ti <- 0
  num_disc_ti <- 0
  num_ties_ti <- 0
  
  for (i in seq_len(n)) {
    if (status[i] != 1) next
    for (j in seq_len(n)) {
      if (i == j) next
      if (time[i] < time[j]) {
        if (risk[i] > risk[j]) {
          num_conc_ti <- num_conc_ti + 1
        } else if (risk[i] < risk[j]) {
          num_disc_ti <- num_disc_ti + 1
        } else {
          num_ties_ti <- num_ties_ti + 1
        }
      }
    }
  }
  
  denom_ti <- num_conc_ti + num_disc_ti + num_ties_ti
  cindex_ti <- if (denom_ti > 0) (num_conc_ti + 0.5 * num_ties_ti) / denom_ti else NA_real_
  
  # Time-dependent C-index
  cindex_td <- tryCatch({
    score_data <- data.frame(time = time, status = status)
    pred_matrix <- matrix(risk, ncol = 1)
    
    evaluation <- riskRegression::Score(
      object = list(Model = pred_matrix),
      formula = Surv(time, status) ~ 1,
      data = score_data,
      times = if (!is.null(horizon)) horizon else median(time[status == 1]),
      summary = "risks",
      metrics = "auc",
      se.fit = FALSE
    )
    
    as.numeric(evaluation$AUC$score$AUC[1])
  }, error = function(e) cindex_ti)
  
  return(list(cindex_td = cindex_td, cindex_ti = cindex_ti))
}

# ranger_predictrisk function
ranger_predictrisk <- function(object, newdata, times) {
  preds <- predict(object, data = newdata, type = "response")
  if (is.null(preds$survival)) {
    stop("ranger prediction did not return survival probabilities")
  }
  
  surv_matrix <- preds$survival
  time_points <- preds$unique.death.times
  closest_idx <- which.min(abs(time_points - times))
  risk_scores <- 1 - surv_matrix[, closest_idx]
  
  return(as.numeric(risk_scores))
}

cat("✓ calculate_cindex() and ranger_predictrisk() defined\n")


✓ calculate_cindex() and ranger_predictrisk() defined


## 3. Monte Carlo Cross-Validation Function

Main function that runs MC-CV for a single method and period.


In [5]:
# Run MC-CV for a single method and time period
run_mc_cv_method <- function(data, method, period_name, mc_splits) {
  
  cat(sprintf("\n=== Running MC-CV for %s (%s) ===\n", method, period_name))
  cat(sprintf("Splits: %d | Train: %.0f%% | Test: %.0f%%\n", 
              n_mc_splits, train_prop * 100, (1 - train_prop) * 100))
  flush.console()
  
  split_ids <- seq_len(n_mc_splits)
  
  # Run splits in parallel with progress bar
  with_progress({
    p <- progressor(steps = n_mc_splits)
    
    results <- future_map(split_ids, function(split_id) {
      p()  # Update progress
      
      # Get train/test data from split
      split <- mc_splits$splits[[split_id]]
      train_data <- rsample::analysis(split)
      test_data <- rsample::assessment(split)
      
      # Train and evaluate
      model <- NULL
      predictions <- NULL
      feature_importance <- NULL
      
      tryCatch({
        if (method == "RSF") {
          model <- ranger(
            Surv(time, status) ~ .,
            data = train_data,
            num.trees = n_trees_rsf,
            importance = "permutation",
            min.node.size = 20,
            splitrule = "extratrees",
            num.random.splits = 10
          )
          
          predictions <- ranger_predictrisk(model, test_data, horizon)
          feature_importance <- model$variable.importance
          
        } else if (method == "AORSF") {
          # Remove constant columns from training data (can occur after train/test split)
          constant_cols <- names(train_data)[sapply(train_data, function(x) {
            if (is.numeric(x)) {
              length(unique(na.omit(x))) <= 1
            } else {
              length(unique(na.omit(x))) <= 1
            }
          })]
          constant_cols <- setdiff(constant_cols, c("time", "status"))
          if (length(constant_cols) > 0) {
            train_data <- train_data %>% select(-any_of(constant_cols))
            test_data <- test_data %>% select(-any_of(constant_cols))
          }
          
          model <- aorsf::orsf(
            data = train_data,
            formula = Surv(time, status) ~ .,
            n_tree = n_trees_aorsf,
            na_action = 'impute_meanmode'
          )
          
          pred_obj <- predict(model, new_data = test_data, 
                              pred_type = 'risk', pred_horizon = horizon)
          predictions <- if (is.matrix(pred_obj)) as.numeric(pred_obj[, 1]) else as.numeric(pred_obj)
          feature_importance <- aorsf::orsf_vi_permute(model)
          
        } else if (method == "CatBoost") {
          # Remove constant columns from training data (can occur after train/test split)
          constant_cols <- names(train_data)[sapply(train_data, function(x) {
            if (is.numeric(x)) {
              length(unique(na.omit(x))) <= 1
            } else {
              length(unique(na.omit(x))) <= 1
            }
          })]
          constant_cols <- setdiff(constant_cols, c("time", "status"))
          if (length(constant_cols) > 0) {
            train_data <- train_data %>% select(-any_of(constant_cols))
            test_data <- test_data %>% select(-any_of(constant_cols))
          }
          
          train_pool <- catboost.load_pool(
            data = train_data %>% select(-time, -status),
            label = train_data$time
          )
          
          test_pool <- catboost.load_pool(
            data = test_data %>% select(-time, -status)
          )
          
          # CatBoost configuration: single-threaded inside each R worker to avoid
          # logger/thread-safety issues, and quiet logging in parallel runs.
          params <- list(
            loss_function  = 'Cox',
            iterations     = 100,
            learning_rate  = 0.1,
            depth          = 6,
            thread_count   = 1,
            logging_level  = 'Silent',
            verbose        = 0L
          )
          
          model <- catboost.train(train_pool, params = params)
          predictions <- catboost.predict(model, test_pool)
          
          # Get feature importance - CatBoost returns a matrix with rownames as feature names
          # IMPORTANT: catboost.get_feature_importance() returns a matrix (not a named vector)
          # - Values are in the first column: importance_matrix[, 1]
          # - Feature names are in rownames: rownames(importance_matrix)
          # Convert to named vector for consistency with RSF and AORSF (which return named vectors directly)
          importance_matrix <- catboost.get_feature_importance(model)
          feature_importance <- as.numeric(importance_matrix[, 1])
          names(feature_importance) <- rownames(importance_matrix)
        }
        
        # Calculate C-index on TEST data
        cindex_result <- calculate_cindex(
          time = test_data$time,
          status = test_data$status,
          risk_scores = predictions,
          horizon = horizon
        )
        
        return(list(
          split_id = split_id,
          cindex_td = cindex_result$cindex_td,
          cindex_ti = cindex_result$cindex_ti,
          feature_importance = feature_importance,
          n_train = nrow(train_data),
          n_test = nrow(test_data),
          success = TRUE
        ))
        
      }, error = function(e) {
        return(list(
          split_id = split_id,
          cindex_td = NA_real_,
          cindex_ti = NA_real_,
          feature_importance = NULL,
          n_train = nrow(train_data),
          n_test = nrow(test_data),
          success = FALSE,
          error = e$message
        ))
      })
    }, .options = furrr_options(
      seed = TRUE,
      packages = c(
        "dplyr", "purrr", "tibble", "rsample",
        "ranger", "aorsf", "catboost",
        "riskRegression", "prodlim"
      )
    ))
  })
  
  # Aggregate results
  successful_splits <- Filter(function(x) x$success, results)
  n_successful <- length(successful_splits)
  
  cat(sprintf("Successful splits: %d / %d\n", n_successful, n_mc_splits))
  flush.console()
  
  # If all splits failed (e.g., CatBoost configuration/data issue),
  # do not stop the whole analysis. Instead, log a warning and return
  # an empty/NA summary so that other methods and periods can continue.
  if (n_successful == 0) {
    error_splits <- Filter(function(x) !is.null(x$error), results)
    if (length(error_splits) > 0) {
      first_error <- error_splits[[1]]$error
      cat(sprintf("WARNING: example error for %s (%s): %s\n",
                  method, period_name, first_error))
    }
    cat(sprintf("WARNING: All MC-CV splits failed for %s (%s). Skipping this method.\n",
                method, period_name))
    flush.console()
    
    return(list(
      method = method,
      period = period_name,
      n_splits = n_mc_splits,
      n_successful = 0,
      cindex_td_mean = NA_real_,
      cindex_td_sd = NA_real_,
      cindex_td_ci_lower = NA_real_,
      cindex_td_ci_upper = NA_real_,
      cindex_ti_mean = NA_real_,
      cindex_ti_sd = NA_real_,
      cindex_ti_ci_lower = NA_real_,
      cindex_ti_ci_upper = NA_real_,
      top_features = numeric(0)
    ))
  }
  
  # Extract C-indexes
  cindex_td_values <- sapply(successful_splits, function(x) x$cindex_td)
  cindex_ti_values <- sapply(successful_splits, function(x) x$cindex_ti)
  
  cindex_td_values <- cindex_td_values[!is.na(cindex_td_values)]
  cindex_ti_values <- cindex_ti_values[!is.na(cindex_ti_values)]
  
  # Aggregate feature importance
  # All methods (RSF, AORSF, CatBoost) return named numeric vectors
  all_feature_names <- unique(unlist(lapply(successful_splits, function(x) {
    if (is.null(x$feature_importance)) return(NULL)
    names(x$feature_importance)
  })))
  
  aggregated_importance <- sapply(all_feature_names, function(feature) {
    importances <- sapply(successful_splits, function(x) {
      if (is.null(x$feature_importance)) return(NA_real_)
      if (feature %in% names(x$feature_importance)) {
        return(as.numeric(x$feature_importance[feature]))
      }
      return(NA_real_)
    })
    mean(importances, na.rm = TRUE)
  })
  
  # Ensure aggregated_importance is a numeric vector
  aggregated_importance <- as.numeric(aggregated_importance)
  names(aggregated_importance) <- all_feature_names
  
  top_features <- sort(aggregated_importance, decreasing = TRUE)[1:min(n_predictors, length(aggregated_importance))]
  
  # Calculate statistics
  results_summary <- list(
    method = method,
    period = period_name,
    n_splits = n_mc_splits,
    n_successful = n_successful,
    cindex_td_mean = mean(cindex_td_values, na.rm = TRUE),
    cindex_td_sd = sd(cindex_td_values, na.rm = TRUE),
    cindex_td_ci_lower = quantile(cindex_td_values, 0.025, na.rm = TRUE),
    cindex_td_ci_upper = quantile(cindex_td_values, 0.975, na.rm = TRUE),
    cindex_ti_mean = mean(cindex_ti_values, na.rm = TRUE),
    cindex_ti_sd = sd(cindex_ti_values, na.rm = TRUE),
    cindex_ti_ci_lower = quantile(cindex_ti_values, 0.025, na.rm = TRUE),
    cindex_ti_ci_upper = quantile(cindex_ti_values, 0.975, na.rm = TRUE),
    top_features = top_features
  )
  
  # Print summary
  cat(sprintf("\n--- Results for %s (%s) ---\n", method, period_name))
  cat(sprintf("Time-Dependent C-Index: %.4f ± %.4f (95%% CI: %.4f - %.4f)\n",
              results_summary$cindex_td_mean,
              results_summary$cindex_td_sd,
              results_summary$cindex_td_ci_lower,
              results_summary$cindex_td_ci_upper))
  cat(sprintf("Time-Independent C-Index: %.4f ± %.4f (95%% CI: %.4f - %.4f)\n",
              results_summary$cindex_ti_mean,
              results_summary$cindex_ti_sd,
              results_summary$cindex_ti_ci_lower,
              results_summary$cindex_ti_ci_upper))
  # Display top 10 features sorted alphabetically for easier comparison
  top10_features <- names(top_features)[1:min(10, length(top_features))]
  top10_features_sorted <- sort(top10_features)
  cat(sprintf("Top 10 features (alphabetical): %s\n", paste(top10_features_sorted, collapse = ", ")))
  flush.console()
  
  return(results_summary)
}

cat("✓ run_mc_cv_method() defined\n")


✓ run_mc_cv_method() defined


## 4. Load Data

In [6]:
# Find and load SAS file
sas_path_local <- here("data", "phts_txpl_ml.sas7bdat")
sas_path_external <- here("graft-loss-parallel-processing", "data", "phts_txpl_ml.sas7bdat")
sas_path_graft_loss <- here("graft-loss", "data", "phts_txpl_ml.sas7bdat")

sas_path <- NULL
for (path in c(sas_path_local, sas_path_external, sas_path_graft_loss)) {
  if (file.exists(path)) {
    sas_path <- path
    break
  }
}

if (is.null(sas_path)) {
  stop("Cannot find phts_txpl_ml.sas7bdat in any location")
}

cat("Loading data from:", sas_path, "\n")

# Load data
phts_base <- haven::read_sas(sas_path) %>%
  filter(TXPL_YEAR >= 2010) %>%
  janitor::clean_names() %>%
  rename(
    outcome_int_graft_loss = int_graft_loss,
    outcome_graft_loss = graft_loss
  ) %>%
  mutate(
    ID = 1:n(),
    across(.cols = where(is.character), ~ ifelse(.x %in% c("", "unknown", "missing"), NA_character_, .x)),
    across(.cols = where(is.character), as.factor),
    tx_mcsd = if ('txnomcsd' %in% names(.)) {
      if_else(txnomcsd == 'yes', 0, 1)
    } else if ('txmcsd' %in% names(.)) {
      txmcsd
    } else {
      NA_real_
    }
  )

cat(sprintf("✓ Loaded data: %d rows, %d columns\n", nrow(phts_base), ncol(phts_base)))


Loading data from: /home/pgx3874/graft-loss/data/phts_txpl_ml.sas7bdat 
✓ Loaded data: 5835 rows, 478 columns


In [7]:
# Define time periods
periods <- list()
periods$original <- phts_base %>% filter(txpl_year >= 2010 & txpl_year <= 2019)
periods$full <- phts_base %>% filter(txpl_year >= 2010)
periods$full_no_covid <- phts_base %>% filter(txpl_year >= 2010 & !(txpl_year >= 2020 & txpl_year <= 2023))

# Print summary
for (period_name in names(periods)) {
  period_data <- periods[[period_name]]
  n_events <- sum(period_data$outcome_graft_loss, na.rm = TRUE)
  event_rate <- 100 * n_events / nrow(period_data)
  
  cat(sprintf("Period: %s | N: %d | Events: %d (%.2f%%)\n", 
              period_name, nrow(period_data), n_events, event_rate))
}

cat("\n✓ Time periods defined\n")


Period: original | N: 4036 | Events: 768 (19.03%)
Period: full | N: 5835 | Events: 939 (16.09%)
Period: full_no_covid | N: 4196 | Events: 774 (18.45%)

✓ Time periods defined


## 5. Run Analysis

Execute MC-CV for all methods and periods.

- **100 splits (default full run):** Typically ~1–2 hours on a 32-core EC2 instance, longer on smaller machines.
- **1000 splits (extended run):** Linearly more expensive; expect roughly 8–10× the 100-split time.

**Note:** You can run each period separately by uncommenting only one `period_name` at a time, or reduce the number of methods/periods to shorten runtime further.


In [ ]:
# Select periods and methods to run
# In DEBUG_MODE, only run original period for speed
period_names <- if (DEBUG_MODE) {
  c("original")  # Debug: just one period (~2-5 min)
} else {
  c("original", "full", "full_no_covid")  # Full: all periods (~30-45 min)
}

# Methods to run. Include CatBoost (with single-threaded, quiet config)
# alongside RSF and AORSF.
method_names <- c("RSF", "CatBoost", "AORSF")

# Store all results
all_results <- list()

# Run analysis
for (period_name in period_names) {
  cat(sprintf("\n========================================\n"))
  cat(sprintf("Processing Period: %s\n", period_name))
  cat(sprintf("========================================\n"))
  flush.console()
  
  # Prepare data
  period_data <- prepare_modeling_data(periods[[period_name]])
  
  cat(sprintf("Period data: %d rows, %d columns\n", nrow(period_data), ncol(period_data)))
  cat(sprintf("Events: %d (%.2f%%)\n", sum(period_data$status), 
              100 * sum(period_data$status) / nrow(period_data)))
  
  # Create MC-CV splits (stratified by outcome)
  cat(sprintf("Creating %d MC-CV splits (stratified)...\n", n_mc_splits))
  flush.console()
  mc_splits <- mc_cv(
    data = period_data,
    prop = train_prop,
    times = n_mc_splits,
    strata = status
  )
  
  # Run each method
  period_results <- list()
  
  for (method in method_names) {
    result <- run_mc_cv_method(period_data, method, period_name, mc_splits)
    period_results[[method]] <- result
    
    # Save top features (sorted alphabetically for easier comparison)
    top_features_df <- tibble(
      feature = names(result$top_features),
      importance = as.numeric(result$top_features),
      cindex_td = result$cindex_td_mean,
      cindex_ti = result$cindex_ti_mean
    ) %>%
      arrange(feature)  # Sort alphabetically for easier visual comparison
    
    output_file <- file.path(output_dir, sprintf("%s_%s_top20.csv", 
                                                  period_name, tolower(method)))
    write_csv(top_features_df, output_file)
    cat(sprintf("✓ Saved: %s\n", basename(output_file)))
  }
  
  all_results[[period_name]] <- period_results
}

cat("\n✓ Analysis complete for all periods!\n")



Processing Period: original
Period data: 4036 rows, 381 columns
Events: 768 (19.03%)
Creating 100 MC-CV splits (stratified)...

=== Running MC-CV for RSF (original) ===
Splits: 100 | Train: 75% | Test: 25%


## 6. Save Summary Results

Create summary tables with C-index comparisons and statistics.


In [ ]:
# Create C-index comparison table
cindex_comparison <- map_df(period_names, function(period) {
  map_df(method_names, function(method) {
    result <- all_results[[period]][[method]]
    tibble(
      period = period,
      method = method,
      cindex_td_mean = result$cindex_td_mean,
      cindex_td_sd = result$cindex_td_sd,
      cindex_td_ci_lower = result$cindex_td_ci_lower,
      cindex_td_ci_upper = result$cindex_td_ci_upper,
      cindex_ti_mean = result$cindex_ti_mean,
      cindex_ti_sd = result$cindex_ti_sd,
      cindex_ti_ci_lower = result$cindex_ti_ci_lower,
      cindex_ti_ci_upper = result$cindex_ti_ci_upper,
      n_splits = result$n_successful
    )
  })
})

write_csv(cindex_comparison, file.path(output_dir, "cindex_comparison_mc_cv.csv"))
cat("✓ Saved: cindex_comparison_mc_cv.csv\n")

# Display the table
print(cindex_comparison)


In [ ]:
# Create summary statistics
summary_stats <- map_df(period_names, function(period) {
  period_data <- periods[[period]]
  tibble(
    period = period,
    n_patients = nrow(period_data),
    n_events = sum(period_data$outcome_graft_loss, na.rm = TRUE),
    event_rate = 100 * sum(period_data$outcome_graft_loss, na.rm = TRUE) / nrow(period_data)
  )
})

write_csv(summary_stats, file.path(output_dir, "summary_statistics_mc_cv.csv"))
cat("✓ Saved: summary_statistics_mc_cv.csv\n")

# Display the table
print(summary_stats)


In [ ]:
# Close parallel processing
plan(sequential)

cat("\n========================================\n")
cat("Analysis Complete!\n")
cat("========================================\n")
cat(sprintf("Output directory: %s\n", output_dir))
cat(sprintf("MC-CV splits: %d\n", n_mc_splits))
cat(sprintf("Train/Test ratio: %.0f/%.0f\n", train_prop * 100, (1 - train_prop) * 100))
cat("\nResults show C-indexes with 95% confidence intervals\n")
cat("based on", n_mc_splits, "independent train/test splits.\n\n")
cat("✓ All files saved successfully!\n")


## 7. Visualize Results

In [ ]:
# Create visualization plots for feature importance and C-index analysis
# Updated to match MC-CV implementation output structure
# Creates: Feature importance heatmap, C-index heatmap, and C-index table

library(tidyverse)
library(ggplot2)
library(here)

# Set output directory (matches notebook output directory)
output_dir <- here("feature_importance", "outputs")
plot_dir <- file.path(output_dir, "plots")
dir.create(plot_dir, showWarnings = FALSE, recursive = TRUE)

# Read MC-CV results
cindex_comparison <- read_csv(file.path(output_dir, "cindex_comparison_mc_cv.csv"))

# Read individual feature files for each method and period
periods <- c("original", "full", "full_no_covid")
methods <- c("rsf", "catboost", "aorsf")

# Load feature importance data
# Map lowercase method names to the format used in cindex_comparison
method_map <- c("rsf" = "RSF", "catboost" = "CatBoost", "aorsf" = "AORSF")

load_features <- function(period, method) {
  file_path <- file.path(output_dir, sprintf("%s_%s_top20.csv", period, method))
  if (file.exists(file_path)) {
    df <- read_csv(file_path) %>%
      mutate(period = period, method = method_map[[method]])
    cat(sprintf("✓ Loaded: %s\n", basename(file_path)))
    return(df)
  } else {
    warning(sprintf("File not found: %s", file_path))
    return(NULL)
  }
}

# Combine all feature files - read all 9 files (3 periods × 3 methods)
cat("\nLoading feature importance files...\n")
all_features <- map_df(periods, function(p) {
  map_df(methods, function(m) {
    load_features(p, m)
  })
}) %>%
  filter(!is.null(feature))

# Verify we loaded all expected files
expected_files <- length(periods) * length(methods)
loaded_count <- length(unique(paste(all_features$period, all_features$method)))
cat(sprintf("\nLoaded %d/%d expected feature files\n", loaded_count, expected_files))
cat(sprintf("Total features loaded: %d\n", nrow(all_features)))
cat(sprintf("Unique features: %d\n", length(unique(all_features$feature))))

# ============================================================================
# 1. FEATURE IMPORTANCE HEATMAP
# ============================================================================

# Get all unique features across all methods and periods
all_unique_features <- unique(all_features$feature)

# Debug: Check what methods and periods are actually in the data
cat("\nDebug: Methods in all_features:", unique(all_features$method), "\n")
cat("Debug: Periods in all_features:", unique(all_features$period), "\n")
cat("Debug: Sample of all_features:\n")
print(head(all_features %>% select(feature, period, method, importance), 10))

# Create a matrix: features (rows) x cohort-method combinations (columns)
# For each feature, get its importance value (or 0 if not in top 20)
feature_matrix <- map_df(all_unique_features, function(feat) {
  map_df(periods, function(per) {
    map_df(methods, function(meth) {
      method_name <- method_map[[meth]]
      feat_data <- all_features %>%
        filter(feature == feat, period == per, method == method_name)
      
      importance_val <- if (nrow(feat_data) > 0) {
        feat_data$importance[1]
      } else {
        0
      }
      
      tibble(
        feature = feat,
        period = per,
        method = method_name,
        importance = importance_val
      )
    })
  })
}) %>%
  mutate(
    cohort_method = paste(period, method, sep = "_"),
    period = factor(period, levels = c("original", "full", "full_no_covid")),
    method = factor(method, levels = c("RSF", "CatBoost", "AORSF"))
  )

# Debug: Check feature_matrix before normalization
cat("\nDebug: Feature matrix summary (before normalization):\n")
cat("Methods in feature_matrix:", unique(feature_matrix$method), "\n")
cat("Non-zero importance values by method:\n")
print(feature_matrix %>% 
      group_by(method) %>% 
      summarise(n_nonzero = sum(importance > 0), 
                max_importance = max(importance), 
                mean_importance = mean(importance), 
                .groups = "drop"))

# Normalize importance values within each method-period combination
# This makes values comparable across different algorithms (RSF, CatBoost, AORSF)
feature_matrix <- feature_matrix %>%
  group_by(period, method) %>%
  mutate(
    importance_normalized = if (max(importance) > 0) {
      (importance - min(importance)) / (max(importance) - min(importance))
    } else {
      0
    }
  ) %>%
  ungroup()

cat("\nDebug: After normalization - importance range by method:\n")
print(feature_matrix %>% 
      group_by(method) %>% 
      summarise(min_imp = min(importance_normalized), 
                max_imp = max(importance_normalized), 
                mean_imp = mean(importance_normalized), 
                .groups = "drop"))

# Determine algorithm ranking by C-index for each period
# Best algorithm gets scale factor 3, second best gets 2, third gets 1
cat("\nDetermining algorithm ranking by C-index for scaling...\n")
algorithm_ranking <- cindex_comparison %>%
  select(period, method, cindex_td_mean) %>%
  group_by(period) %>%
  arrange(desc(cindex_td_mean)) %>%
  mutate(
    rank = row_number(),
    scale_factor = case_when(
      rank == 1 ~ 3,  # Best algorithm
      rank == 2 ~ 2,  # Second best algorithm
      rank == 3 ~ 1   # Third algorithm (no scaling)
    )
  ) %>%
  ungroup() %>%
  select(period, method, scale_factor)

cat("Algorithm ranking and scale factors:\n")
print(algorithm_ranking)

# Apply scaling to normalized feature importance
feature_matrix <- feature_matrix %>%
  left_join(algorithm_ranking, by = c("period", "method")) %>%
  mutate(
    importance_scaled = importance_normalized * scale_factor,
    importance = importance_scaled  # Use scaled values for heatmap
  ) %>%
  select(-importance_scaled)

cat("\nDebug: After scaling - importance range by method:\n")
print(feature_matrix %>% 
      group_by(method) %>% 
      summarise(min_imp = min(importance), 
                max_imp = max(importance), 
                mean_imp = mean(importance), 
                .groups = "drop"))

# Create heatmap
# Order features by total importance across all cohort-method combinations
feature_order <- feature_matrix %>%
  group_by(feature) %>%
  summarise(total_importance = sum(importance), .groups = "drop") %>%
  arrange(desc(total_importance)) %>%
  pull(feature)

feature_matrix <- feature_matrix %>%
  mutate(feature = factor(feature, levels = feature_order))

# Create cohort_method labels with better formatting
feature_matrix <- feature_matrix %>%
  mutate(
    cohort_label = case_when(
      period == "original" ~ "Original",
      period == "full" ~ "Full",
      period == "full_no_covid" ~ "Full No COVID"
    ),
    cohort_method_label = paste(cohort_label, method, sep = "\n")
  )

p1 <- ggplot(feature_matrix, aes(x = cohort_method_label, y = feature, fill = importance)) +
  geom_tile(color = "white", linewidth = 0.1) +
  scale_fill_gradient(low = "orange", high = "darkblue",
                      name = "Importance") +
  labs(title = "Feature Importance Heatmap by Cohort and Algorithm",
       x = "Cohort × Algorithm", y = "Feature") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 0, hjust = 0.5, size = 10),
    axis.text.y = element_text(size = 8),
    axis.title = element_text(size = 12, face = "bold"),
    plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
    legend.position = "right",
    panel.grid = element_blank()
  )

ggsave(file.path(plot_dir, "feature_importance_heatmap.png"), p1, 
       width = 12, height = max(16, length(all_unique_features) * 0.3), dpi = 300, limitsize = FALSE)

# ============================================================================
# 2. C-INDEX HEATMAP
# ============================================================================

# Prepare data for C-index heatmap (both time-dependent and time-independent)
cindex_heatmap_data <- bind_rows(
  cindex_comparison %>%
    select(period, method, cindex_td_mean) %>%
    rename(cindex = cindex_td_mean) %>%
    mutate(cindex_type = "Time-Dependent"),
  cindex_comparison %>%
    select(period, method, cindex_ti_mean) %>%
    rename(cindex = cindex_ti_mean) %>%
    mutate(cindex_type = "Time-Independent")
) %>%
  mutate(
    period = factor(period, levels = c("original", "full", "full_no_covid")),
    method = factor(method, levels = c("RSF", "CatBoost", "AORSF")),
    cohort_label = case_when(
      period == "original" ~ "Original",
      period == "full" ~ "Full",
      period == "full_no_covid" ~ "Full No COVID"
    )
  )

p2 <- ggplot(cindex_heatmap_data, aes(x = method, y = cohort_label, fill = cindex)) +
  geom_tile(color = "white", linewidth = 0.5) +
  geom_text(aes(label = sprintf("%.3f", cindex)), color = "black", size = 4, fontface = "bold") +
  scale_fill_gradient2(low = "red", mid = "white", high = "green",
                       midpoint = 0.5,
                       name = "C-index") +
  facet_wrap(~cindex_type, ncol = 2) +
  labs(title = "Concordance Index Heatmap by Cohort and Algorithm (MC-CV)",
       x = "Algorithm", y = "Cohort") +
  theme_minimal() +
  theme(
    axis.text = element_text(size = 11),
    axis.title = element_text(size = 12, face = "bold"),
    plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
    legend.position = "right",
    strip.text = element_text(size = 12, face = "bold"),
    panel.grid = element_blank()
  )

ggsave(file.path(plot_dir, "cindex_heatmap.png"), p2, width = 12, height = 6, dpi = 300)

# ============================================================================
# 3. SCALED FEATURE IMPORTANCE BAR CHART
# ============================================================================

cat("\nCreating scaled feature importance bar chart...\n")

# Aggregate scaled importance by feature across all periods and methods
# Sum the scaled importance values for each feature
scaled_feature_importance <- feature_matrix %>%
  group_by(feature) %>%
  summarise(
    total_scaled_importance = sum(importance_normalized * scale_factor, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  arrange(desc(total_scaled_importance)) %>%
  # Get top 20 features
  slice_head(n = 20)

# Order features by total scaled importance
scaled_feature_importance <- scaled_feature_importance %>%
  mutate(feature = factor(feature, levels = rev(scaled_feature_importance$feature)))

p3 <- ggplot(scaled_feature_importance, aes(x = feature, y = total_scaled_importance)) +
  geom_bar(stat = "identity", fill = "steelblue", alpha = 0.8) +
  coord_flip() +
  labs(
    title = "Scaled Feature Importance (Top 20 Features)",
    subtitle = "Importance scaled by algorithm performance: Best C-index (×3), Second best (×2), Third (×1)",
    x = "Feature",
    y = "Scaled Normalized Importance"
  ) +
  theme_minimal() +
  theme(
    axis.text = element_text(size = 10),
    axis.title = element_text(size = 12, face = "bold"),
    plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
    plot.subtitle = element_text(size = 11, hjust = 0.5),
    panel.grid.major.y = element_blank(),
    panel.grid.minor = element_blank()
  )

ggsave(file.path(plot_dir, "scaled_feature_importance_bar_chart.png"), p3, 
       width = 12, height = 10, dpi = 300)
cat("✓ Saved: scaled_feature_importance_bar_chart.png\n")

# ============================================================================
# 4. C-INDEX TABLE
# ============================================================================

# Create formatted table with both C-index types and confidence intervals
cindex_table <- cindex_comparison %>%
  mutate(
    period_label = case_when(
      period == "original" ~ "Original",
      period == "full" ~ "Full",
      period == "full_no_covid" ~ "Full No COVID"
    ),
    # Format C-index with CI
    cindex_td_formatted = sprintf("%.3f (%.3f-%.3f)", 
                                  cindex_td_mean, cindex_td_ci_lower, cindex_td_ci_upper),
    cindex_ti_formatted = sprintf("%.3f (%.3f-%.3f)", 
                                   cindex_ti_mean, cindex_ti_ci_lower, cindex_ti_ci_upper)
  ) %>%
  select(period_label, method, cindex_td_formatted, cindex_ti_formatted, n_splits) %>%
  arrange(period_label, method) %>%
  rename(
    Cohort = period_label,
    Algorithm = method,
    `Time-Dependent C-index (95% CI)` = cindex_td_formatted,
    `Time-Independent C-index (95% CI)` = cindex_ti_formatted,
    `N Splits` = n_splits
  )

# Save as CSV
write_csv(cindex_table, file.path(plot_dir, "cindex_table.csv"))

# Also create a formatted text table for display
cat("\n========================================\n")
cat("Concordance Index Table\n")
cat("========================================\n")
print(cindex_table)
cat("\n")

# Save summary
cat("Plots saved to:", plot_dir, "\n")
cat("Created visualizations:\n")
cat("  1. feature_importance_heatmap.png - Feature importance by cohort and algorithm (scaled by C-index)\n")
cat("  2. cindex_heatmap.png - Concordance index by cohort and algorithm\n")
cat("  3. scaled_feature_importance_bar_chart.png - Bar chart of scaled feature importance (top 20)\n")
cat("  4. cindex_table.csv - Concordance index table with confidence intervals\n")
cat("\nAll visualizations use MC-CV results with 95% confidence intervals.\n")
cat("Feature importance values are normalized within each method-period combination,\n")
cat("then scaled by algorithm performance: Best C-index algorithm (×3), Second best (×2), Third (×1).\n")



## 8. Sync Results and Code to S3

Sync output files and code (notebook + R script) to S3 bucket. 
- Outputs: CSV results files
- Code: Notebook and R script for reproducibility

In [ ]:
# Sync outputs and code to S3
# On EC2, we're in the feature_importance directory
s3_bucket <- "s3://uva-private-data-lake/graft-loss/feature_importance/"

# Find AWS CLI (check common locations - EC2 typically has it in /usr/local/bin or /usr/bin)
aws_cmd <- Sys.which("aws")
if (aws_cmd == "") {
  # Try common EC2 installation paths
  aws_paths <- c(
    "/usr/local/bin/aws",
    "/usr/bin/aws",
    "/home/ec2-user/.local/bin/aws"
  )
  aws_cmd <- NULL
  for (path in aws_paths) {
    if (file.exists(path)) {
      aws_cmd <- path
      break
    }
  }
  if (is.null(aws_cmd)) {
    stop("AWS CLI not found. Please install AWS CLI or ensure it's in your PATH.")
  }
}

cat("Syncing outputs and code to S3...\n")
cat("Source: feature_importance/ directory\n")
cat("Destination:", s3_bucket, "\n")
cat("AWS CLI:", aws_cmd, "\n\n")

# Get current directory (should be feature_importance)
current_dir <- getwd()
if (!grepl("feature_importance", current_dir)) {
  warning("Current directory doesn't appear to be feature_importance. Double-check sync destination.")
}

# Sync feature_importance directory (includes outputs/ and code files)
# Exclude temporary files, checkpoints, and unnecessary directories
# Note: --delete flag removed for safety (won't delete files in S3 that don't exist locally)
# This will sync: outputs/*, *.ipynb, *.R, README*.md (and exclude everything else unwanted)
sync_cmd <- sprintf(
  '"%s" s3 sync "%s" %s --exclude "*checkpoint*" --exclude "*.tmp" --exclude "*.ipynb_checkpoints/*" --exclude "*.RData" --exclude "*.Rhistory" --exclude ".Rproj.user/*" --exclude "catboost_info/*" --exclude "*.log"',
  aws_cmd,
  current_dir,
  s3_bucket
)

cat("Running:", sync_cmd, "\n\n")
result <- system(sync_cmd)

if (result == 0) {
  cat("✓ Successfully synced outputs and code to S3\n")
  cat("  - Outputs:", file.path(output_dir), "\n")
  cat("  - Code: *.ipynb, *.R, README*.md\n")
} else {
  warning(sprintf("S3 sync returned exit code %d. Check AWS credentials and permissions.", result))
}


## 9. Shutdown EC2

In [ ]:

# Shutdown EC2 instance after analysis completes
# Set SHUTDOWN_EC2 = TRUE to enable, FALSE to disable
SHUTDOWN_EC2 <- TRUE  # Change to TRUE to enable auto-shutdown

if (SHUTDOWN_EC2) {
  cat("\n========================================\n")
  cat("Shutting down EC2 instance...\n")
  cat("========================================\n")
  
  # Get instance ID from EC2 metadata service
  instance_id <- tryCatch({
    system("curl -s http://169.254.169.254/latest/meta-data/instance-id", intern = TRUE)
  }, error = function(e) {
    cat("Warning: Could not retrieve instance ID from metadata service.\n")
    cat("If running on EC2, check that metadata service is accessible.\n")
    return(NULL)
  })
  
  if (!is.null(instance_id) && length(instance_id) > 0 && nchar(instance_id[1]) > 0) {
    instance_id <- instance_id[1]
    cat(sprintf("Instance ID: %s\n", instance_id))
    
    # Find AWS CLI
    aws_cmd <- Sys.which("aws")
    if (aws_cmd == "") {
      aws_paths <- c(
        "/usr/local/bin/aws",
        "/usr/bin/aws",
        "/home/ec2-user/.local/bin/aws"
      )
      aws_cmd <- NULL
      for (path in aws_paths) {
        if (file.exists(path)) {
          aws_cmd <- path
          break
        }
      }
    }
    
    if (!is.null(aws_cmd) && aws_cmd != "") {
      # Stop the instance (use terminate-instances for permanent deletion)
      shutdown_cmd <- sprintf(
        '"%s" ec2 stop-instances --instance-ids %s',
        aws_cmd,
        instance_id
      )
      
      cat("Running:", shutdown_cmd, "\n")
      result <- system(shutdown_cmd)
      
      if (result == 0) {
        cat("✓ EC2 instance stop command sent successfully\n")
        cat("Instance will stop in a few moments.\n")
        cat("Note: This is a STOP (not terminate), so you can restart it later.\n")
      } else {
        warning(sprintf("EC2 stop command returned exit code %d. Check AWS credentials and permissions.", result))
      }
    } else {
      cat("Warning: AWS CLI not found. Cannot shutdown instance.\n")
      cat("Install AWS CLI or ensure it's in your PATH.\n")
    }
  } else {
    cat("Warning: Could not determine instance ID. Skipping shutdown.\n")
    cat("If you want to shutdown manually, use:\n")
    cat("  aws ec2 stop-instances --instance-ids <your-instance-id>\n")
  }
} else {
  cat("\n========================================\n")
  cat("EC2 Auto-Shutdown: DISABLED\n")
  cat("========================================\n")
  cat("To enable auto-shutdown, set SHUTDOWN_EC2 = TRUE in this cell.\n")
  cat("Instance will continue running.\n")
}
